### 01. Import Dependecies

In [1]:
import pandas as pd

### 02. Loading Data

In [2]:
df = pd.read_csv("data/processed/handled_missing_values.csv")
df.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


In [3]:
df_ip = pd.read_csv('data/raw/IpAddress_to_Country.csv')
df_ip.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


### 03. Feature Engineering

In [4]:
df['signup_time'] = pd.to_datetime(df['signup_time'])
df['purchase_time'] = pd.to_datetime(df['purchase_time'])
df['account_age_minutes'] = (df['purchase_time'] - df['signup_time']).dt.total_seconds() / 60

In [5]:
df['device_count'] = df.groupby('device_id')['device_id'].transform('count')

In [6]:
df['user_count_per_device'] = df.groupby('device_id')['user_id'].transform('nunique')

In [7]:
df.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,account_age_minutes,device_count,user_count_per_device
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,75111.366667,1,1
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,299.066667,1,1
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,0.016667,12,12
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,8201.416667,1,1
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,72691.016667,1,1


In [8]:
import pandas as pd

# 1. IP ලිපින පූර්ණ සංඛ්‍යා (Int64) බවට හරවන්න (මෙතන වැරදුනොත් තමයි NaN එන්නේ)
df['ip_address'] = df['ip_address'].astype('int64')
df_ip['lower_bound_ip_address'] = df_ip['lower_bound_ip_address'].astype('int64')
df_ip['upper_bound_ip_address'] = df_ip['upper_bound_ip_address'].astype('int64')

# 2. අනිවාර්යයෙන්ම Sort කරන්න (පිළිවෙළ වැදගත්)
df = df.sort_values('ip_address')
df_ip = df_ip.sort_values('lower_bound_ip_address')

# 3. Merge කිරීම
df_merged = pd.merge_asof(
    df, 
    df_ip, 
    left_on='ip_address', 
    right_on='lower_bound_ip_address',
    direction='backward' # ආසන්නතම අඩු අගය බලන්න
)

# 4. වැදගත්ම දේ: IP එක upper bound එකට වඩා වැඩි නම් ඒවා අයින් කරන්න (Null කරන්න)
# මොකද merge_asof බලන්නේ lower bound එක විතරයි.
df_merged.loc[df_merged['ip_address'] > df_merged['upper_bound_ip_address'], 'country'] = pd.NA

# 5. ඉතිරි වන NaN (ඇත්තටම රටක් නැති ඒවා) වලට 'Unknown' කියලා නමක් දෙන්න
df_merged['country'] = df_merged['country'].fillna('Unknown')

In [9]:
df_merged.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,account_age_minutes,device_count,user_count_per_device,lower_bound_ip_address,upper_bound_ip_address,country
0,62421,2015-02-16 00:17:05,2015-03-08 10:00:39,46,ZCLZTAJPCRAQX,Direct,Safari,M,36,52093,0,29383.566667,1,1,NaN,NaN,Unknown
1,173212,2015-03-08 04:03:22,2015-03-20 17:23:45,33,YFGYOALADBHLT,Ads,IE,F,30,93447,0,18080.383333,1,1,NaN,NaN,Unknown
2,242286,2015-05-17 16:45:54,2015-05-26 08:54:34,33,QZNVQTUITFTHH,Direct,FireFox,F,32,105818,0,12488.666667,1,1,NaN,NaN,Unknown
3,370003,2015-03-03 19:58:39,2015-05-28 21:09:13,33,PIBUQMBIELMMG,Ads,IE,M,40,117566,0,123910.566667,1,1,NaN,NaN,Unknown
4,119824,2015-03-20 00:31:27,2015-04-05 07:31:46,55,WFIIFCPIOGMHT,Ads,Safari,M,38,131423,0,23460.316667,1,1,NaN,NaN,Unknown
